In [2]:
from pathlib import Path
from netCDF4 import Dataset as NetCDFFile 
import matplotlib.pyplot as plt
import numpy as np
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %matplotlib inline
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
from mpl_toolkits.axes_grid1 import AxesGrid, ImageGrid
import pandas as pd 
import geopandas as gpd
import pygeos
import seaborn as sns
import xarray as xr
import rioxarray as rio
from matplotlib.colors import LinearSegmentedColormap
import datetime
from datetime import timedelta
from scipy.spatial.distance import cdist
from math import dist
from scipy.stats import ttest_ind
from geopy import distance
from scipy import stats
from matplotlib.gridspec import GridSpec
from pylr2 import regress2
from matplotlib.dates import (AutoDateLocator, YearLocator, MonthLocator,
                              DayLocator, WeekdayLocator, HourLocator,
                              MinuteLocator, SecondLocator, MicrosecondLocator,
                              RRuleLocator, rrulewrapper, MONTHLY,
                              MO, TU, WE, TH, FR, SA, SU, DateFormatter,
                              AutoDateFormatter, ConciseDateFormatter)
from matplotlib.ticker import MaxNLocator,FixedLocator
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize
from scipy.stats import gaussian_kde

# import shutil

# Get hour Pandora

In [3]:
lisPD = []
# total VCD
for path in Path('./fus5/').rglob('*fus5*.txt'):
    lisPD.append(path)
print(len(lisPD))
lisPD[:5]

19


[PosixPath('fus5/Pandora149s1_Seoul-SNU_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora54s1_Seoul_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora77s1_Singapore-NUS_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora217s1_Dalanzadgad_L2_rfus5p1-8.txt'),
 PosixPath('fus5/Pandora20s1_Busan_L2_rfus5p1-8.txt')]

In [5]:
pandora = pd.DataFrame(columns = ['Time', 'L1QA', 'L2fitQA', 'VCD(moles/m2)', 'TotalUncertainty','L1Height','L2HCHOQA',
       'RmsUncertainty', 'Status', 'Location', 'Country', 'Lat', 'Lon',
       'Altitude', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Datetime','Localtime'])

for i in lisPD:
    cols = list(range(1,53))

    # Read txt to df
    PD = pd.read_csv(i,skiprows=74, encoding='unicode_escape',delimiter=' ',header=0,names=cols)
    PD = PD.iloc[:, [0,29,32,38,42,43,50,35]]

    cols2 = ['Time','L1QA','L2fitQA','VCD(moles/m2)','TotalUncertainty','RmsUncertainty','L1Height','L2HCHOQA']
    PD.columns = cols2
    # Read Attributes
    file = open(i, encoding='unicode_escape')
    content = file.readlines()
    status = content[4][43:-1]
    location = content[13][21:-1]
    country = content[14][21:-1]
    lat = float(content[15][25:-1])
    lon = float(content[16][26:-1])
    altitude = float(content[17][23:-1])
    
    timezone = int(round(lon/15))
    
    PD['Status'] = status
    PD['Location'] = location
    PD['Country'] = country
    PD['Lat'] = lat
    PD['Lon'] = lon
    PD['Altitude'] = altitude
    
    yearLis = []
    monthLis = []
    dayLis = []
    hourLis = []
    minuteLis = []
    datetimeLis = []
    for j in PD['Time']:
        yearLis.append(j[:4])
        monthLis.append(j[4:6])
        dayLis.append(j[6:8])
        hourLis.append(j[9:11])
        minuteLis.append(j[11:13])
        datetime_obj = datetime.datetime(int(j[:4]), int(j[4:6]), int(j[6:8]), int(j[9:11]), int(j[11:13]))
        datetimeLis.append(datetime_obj)
    PD['Year'] = yearLis
    PD['Month'] = monthLis
    PD['Day'] = dayLis
    PD['Hour'] = hourLis
    PD['Minute'] = minuteLis
    PD['Datetime'] = datetimeLis
    PD['Localtime'] = PD['Datetime'] + pd.Timedelta(hours=timezone)
    pandora = pd.concat([pandora,PD])
pandora

,Time,L1QA,L2fitQA,VCD(moles/m2),TotalUncertainty,L1Height,L2HCHOQA,RmsUncertainty,Status,Location,...,Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime
0,20190207T023554.1Z,10,12,-0.001059,0.000080,0.000,22,0.000242,official,Seoul-SNU,...,37.4580,126.951,116.0,2019,02,07,02,35,2019-02-07 02:35:00,2019-02-07 10:35:00
1,20190207T023727.4Z,10,12,-0.001090,0.000080,0.000,22,0.000244,official,Seoul-SNU,...,37.4580,126.951,116.0,2019,02,07,02,37,2019-02-07 02:37:00,2019-02-07 10:37:00
2,20190207T023857.9Z,10,12,-0.001040,0.000079,0.000,22,0.000247,official,Seoul-SNU,...,37.4580,126.951,116.0,2019,02,07,02,38,2019-02-07 02:38:00,2019-02-07 10:38:00
3,20190207T024026.0Z,10,12,-0.001010,0.000080,0.000,22,0.000247,official,Seoul-SNU,...,37.4580,126.951,116.0,2019,02,07,02,40,2019-02-07 02:40:00,2019-02-07 10:40:00
4,20190207T024154.9Z,10,12,-0.001013,0.000079,0.000,22,0.000251,official,Seoul-SNU,...,37.4580,126.951,116.0,2019,02,07,02,41,2019-02-07 02:41:00,2019-02-07 10:41:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60798,20240213T065347.1Z,10,11,0.000108,0.000008,0.004,11,0.000018,unvalidated,Tsukuba-NIES,...,36.0513,140.121,45.0,2024,02,13,06,53,2024-02-13 06:53:00,2024-02-13 15:53:00
60799,20240213T070424.2Z,10,11,0.000116,0.000008,0.007,11,0.000017,unvalidated,Tsukuba-NIES,...,36.0513,140.121,45.0,2024,02,13,07,04,2024-02-13 07:04:00,2024-02-13 16:04:00
60800,20240213T071418.1Z,10,11,0.000119,0.000008,0.010,11,0.000018,unvalidated,Tsukuba-NIES,...,36.0513,140.121,45.0,2024,02,13,07,14,2024-02-13 07:14:00,2024-02-13 16:14:00
60801,20240213T073216.1Z,10,12,0.000116,0.000011,0.029,12,0.000029,unvalidated,Tsukuba-NIES,...,36.0513,140.121,45.0,2024,02,13,07,32,2024-02-13 07:32:00,2024-02-13 16:32:00


In [7]:
pandora = pandora[pandora['Year'].isin(['2021','2022','2023'])]
print(len(pandora))
pandora.head()

793864


,Time,L1QA,L2fitQA,VCD(moles/m2),TotalUncertainty,L1Height,L2HCHOQA,RmsUncertainty,Status,Location,...,Lat,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime
38185,20210101T000840.4Z,12,12,0.000134,0.000054,0.007,12,0.000073,official,Seoul-SNU,...,37.458,126.951,116.0,2021,01,01,00,08,2021-01-01 00:08:00,2021-01-01 08:08:00
38186,20210101T001442.7Z,10,11,0.000140,0.000016,0.006,12,0.000018,official,Seoul-SNU,...,37.458,126.951,116.0,2021,01,01,00,14,2021-01-01 00:14:00,2021-01-01 08:14:00
38187,20210101T002425.7Z,10,11,0.000145,0.000017,0.004,12,0.000019,official,Seoul-SNU,...,37.458,126.951,116.0,2021,01,01,00,24,2021-01-01 00:24:00,2021-01-01 08:24:00
38188,20210101T003434.7Z,10,11,0.000127,0.000019,0.003,12,0.000018,official,Seoul-SNU,...,37.458,126.951,116.0,2021,01,01,00,34,2021-01-01 00:34:00,2021-01-01 08:34:00
38189,20210101T004045.3Z,12,12,0.000181,0.000052,0.003,12,0.000055,official,Seoul-SNU,...,37.458,126.951,116.0,2021,01,01,00,40,2021-01-01 00:40:00,2021-01-01 08:40:00


In [8]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))
pandora['Datetime1']=pandora['Datetime'].map(hour_rounder)
pandora.head()

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_62208/33972301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora['Datetime1']=pandora['Datetime'].map(hour_rounder)


,Time,L1QA,L2fitQA,VCD(moles/m2),TotalUncertainty,L1Height,L2HCHOQA,RmsUncertainty,Status,Location,...,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime,Datetime1
38185,20210101T000840.4Z,12,12,0.000134,0.000054,0.007,12,0.000073,official,Seoul-SNU,...,126.951,116.0,2021,01,01,00,08,2021-01-01 00:08:00,2021-01-01 08:08:00,2021-01-01 00:00:00
38186,20210101T001442.7Z,10,11,0.000140,0.000016,0.006,12,0.000018,official,Seoul-SNU,...,126.951,116.0,2021,01,01,00,14,2021-01-01 00:14:00,2021-01-01 08:14:00,2021-01-01 00:00:00
38187,20210101T002425.7Z,10,11,0.000145,0.000017,0.004,12,0.000019,official,Seoul-SNU,...,126.951,116.0,2021,01,01,00,24,2021-01-01 00:24:00,2021-01-01 08:24:00,2021-01-01 00:00:00
38188,20210101T003434.7Z,10,11,0.000127,0.000019,0.003,12,0.000018,official,Seoul-SNU,...,126.951,116.0,2021,01,01,00,34,2021-01-01 00:34:00,2021-01-01 08:34:00,2021-01-01 01:00:00
38189,20210101T004045.3Z,12,12,0.000181,0.000052,0.003,12,0.000055,official,Seoul-SNU,...,126.951,116.0,2021,01,01,00,40,2021-01-01 00:40:00,2021-01-01 08:40:00,2021-01-01 01:00:00


In [14]:
# If we only use '0', which is 'assured high quality', the available data is too little
pandora2 = pandora
# pandora2 = pandora2[pandora2['L1QA'].isin([0])]
# pandora2 = pandora2[pandora2['L2fitQA'].isin([0])]
pandora2 = pandora2[pandora2['L2HCHOQA'].isin([0])]
pandora2

,Time,L1QA,L2fitQA,VCD(moles/m2),TotalUncertainty,L1Height,L2HCHOQA,RmsUncertainty,Status,Location,...,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime,Datetime1
88,20230616T022223.6Z,10,10,0.000247,0.000013,0.0,0,0.000014,official,Singapore-NUS,...,103.771,77.0,2023,06,16,02,22,2023-06-16 02:22:00,2023-06-16 09:22:00,2023-06-16 02:00:00
89,20230616T022518.9Z,10,10,0.000259,0.000013,0.0,0,0.000014,official,Singapore-NUS,...,103.771,77.0,2023,06,16,02,25,2023-06-16 02:25:00,2023-06-16 09:25:00,2023-06-16 02:00:00
94,20230616T024001.3Z,10,10,0.000243,0.000013,0.0,0,0.000015,official,Singapore-NUS,...,103.771,77.0,2023,06,16,02,40,2023-06-16 02:40:00,2023-06-16 09:40:00,2023-06-16 03:00:00
96,20230616T024602.1Z,10,10,0.000250,0.000014,0.0,0,0.000015,official,Singapore-NUS,...,103.771,77.0,2023,06,16,02,46,2023-06-16 02:46:00,2023-06-16 09:46:00,2023-06-16 03:00:00
97,20230616T025233.3Z,10,10,0.000245,0.000014,0.0,0,0.000014,official,Singapore-NUS,...,103.771,77.0,2023,06,16,02,52,2023-06-16 02:52:00,2023-06-16 09:52:00,2023-06-16 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210,20220525T035631.4Z,10,10,0.000182,0.000013,0.0,0,0.000023,unvalidated,Tsukuba-NIES,...,140.121,45.0,2022,05,25,03,56,2022-05-25 03:56:00,2022-05-25 12:56:00,2022-05-25 04:00:00
20212,20220525T040608.9Z,10,10,0.000189,0.000013,0.0,0,0.000023,unvalidated,Tsukuba-NIES,...,140.121,45.0,2022,05,25,04,06,2022-05-25 04:06:00,2022-05-25 13:06:00,2022-05-25 04:00:00
20213,20220525T040635.9Z,10,10,0.000193,0.000014,0.0,0,0.000024,unvalidated,Tsukuba-NIES,...,140.121,45.0,2022,05,25,04,06,2022-05-25 04:06:00,2022-05-25 13:06:00,2022-05-25 04:00:00
20214,20220525T041131.0Z,10,10,0.000205,0.000013,0.0,0,0.000022,unvalidated,Tsukuba-NIES,...,140.121,45.0,2022,05,25,04,11,2022-05-25 04:11:00,2022-05-25 13:11:00,2022-05-25 04:00:00


In [16]:
# Thus, we include "10", which is "unassured high quality"
pandora2 = pandora
# pandora2 = pandora2[pandora2['L1QA'].isin([0,10])]
# pandora2 = pandora2[pandora2['L2fitQA'].isin([0,10])]
pandora2 = pandora2[pandora2['L2HCHOQA'].isin([0,10])]
pandora2

,Time,L1QA,L2fitQA,VCD(moles/m2),TotalUncertainty,L1Height,L2HCHOQA,RmsUncertainty,Status,Location,...,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime,Datetime1
38261,20210102T031825.0Z,10,10,0.000186,0.000030,0.000,10,0.000022,official,Seoul-SNU,...,126.951,116.0,2021,01,02,03,18,2021-01-02 03:18:00,2021-01-02 11:18:00,2021-01-02 03:00:00
38264,20210102T033433.2Z,10,10,0.000192,0.000030,0.000,10,0.000023,official,Seoul-SNU,...,126.951,116.0,2021,01,02,03,34,2021-01-02 03:34:00,2021-01-02 11:34:00,2021-01-02 04:00:00
38274,20210102T043817.3Z,10,10,0.000183,0.000028,0.001,10,0.000022,official,Seoul-SNU,...,126.951,116.0,2021,01,02,04,38,2021-01-02 04:38:00,2021-01-02 12:38:00,2021-01-02 05:00:00
38275,20210102T044338.2Z,10,10,0.000184,0.000028,0.001,10,0.000021,official,Seoul-SNU,...,126.951,116.0,2021,01,02,04,43,2021-01-02 04:43:00,2021-01-02 12:43:00,2021-01-02 05:00:00
38314,20210103T020311.0Z,10,10,0.000186,0.000026,0.001,10,0.000020,official,Seoul-SNU,...,126.951,116.0,2021,01,03,02,03,2021-01-03 02:03:00,2021-01-03 10:03:00,2021-01-03 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58330,20231230T033611.6Z,10,10,0.000147,0.000008,0.001,10,0.000023,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,03,36,2023-12-30 03:36:00,2023-12-30 12:36:00,2023-12-30 04:00:00
58331,20231230T034133.1Z,10,10,0.000150,0.000008,0.001,10,0.000022,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,03,41,2023-12-30 03:41:00,2023-12-30 12:41:00,2023-12-30 04:00:00
58332,20231230T035210.4Z,10,10,0.000151,0.000008,0.001,10,0.000023,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,03,52,2023-12-30 03:52:00,2023-12-30 12:52:00,2023-12-30 04:00:00
58334,20231230T040842.9Z,10,10,0.000159,0.000008,0.001,10,0.000022,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,04,08,2023-12-30 04:08:00,2023-12-30 13:08:00,2023-12-30 04:00:00


In [17]:
# We only use data that is close to GEMS observation time
pandora2['Minute'] = pandora2['Minute'].astype(int)
pandora3 = pandora2[(pandora2['Minute']<=15)|(pandora2['Minute']>=45)].copy()
pandora3

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_62208/3185678102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora2['Minute'] = pandora2['Minute'].astype(int)


,Time,L1QA,L2fitQA,VCD(moles/m2),TotalUncertainty,L1Height,L2HCHOQA,RmsUncertainty,Status,Location,...,Lon,Altitude,Year,Month,Day,Hour,Minute,Datetime,Localtime,Datetime1
38314,20210103T020311.0Z,10,10,0.000186,0.000026,0.001,10,0.000020,official,Seoul-SNU,...,126.951,116.0,2021,01,03,02,3,2021-01-03 02:03:00,2021-01-03 10:03:00,2021-01-03 02:00:00
38478,20210106T020638.3Z,10,10,0.000173,0.000027,0.001,10,0.000020,official,Seoul-SNU,...,126.951,116.0,2021,01,06,02,6,2021-01-06 02:06:00,2021-01-06 10:06:00,2021-01-06 02:00:00
38479,20210106T021200.2Z,10,10,0.000172,0.000027,0.001,10,0.000020,official,Seoul-SNU,...,126.951,116.0,2021,01,06,02,12,2021-01-06 02:12:00,2021-01-06 10:12:00,2021-01-06 02:00:00
38485,20210106T024835.3Z,10,10,0.000178,0.000029,0.001,10,0.000021,official,Seoul-SNU,...,126.951,116.0,2021,01,06,02,48,2021-01-06 02:48:00,2021-01-06 10:48:00,2021-01-06 03:00:00
38486,20210106T025817.3Z,10,10,0.000177,0.000030,0.001,10,0.000021,official,Seoul-SNU,...,126.951,116.0,2021,01,06,02,58,2021-01-06 02:58:00,2021-01-06 10:58:00,2021-01-06 03:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58325,20231230T030925.5Z,10,10,0.000131,0.000009,0.000,10,0.000022,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,03,9,2023-12-30 03:09:00,2023-12-30 12:09:00,2023-12-30 03:00:00
58326,20231230T031445.8Z,10,10,0.000130,0.000008,0.000,10,0.000023,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,03,14,2023-12-30 03:14:00,2023-12-30 12:14:00,2023-12-30 03:00:00
58332,20231230T035210.4Z,10,10,0.000151,0.000008,0.001,10,0.000023,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,03,52,2023-12-30 03:52:00,2023-12-30 12:52:00,2023-12-30 04:00:00
58334,20231230T040842.9Z,10,10,0.000159,0.000008,0.001,10,0.000022,unvalidated,Tsukuba-NIES,...,140.121,45.0,2023,12,30,04,8,2023-12-30 04:08:00,2023-12-30 13:08:00,2023-12-30 04:00:00


In [18]:
pandora3 = pandora3[[  'VCD(moles/m2)',  'Location', 
       'Lat', 'Lon', 'Datetime1']]
pandora3['Year'] = pandora3['Datetime1'].dt.year
pandora3['Month'] = pandora3['Datetime1'].dt.month
pandora3['Day'] = pandora3['Datetime1'].dt.day
pandora3['Hour'] = pandora3['Datetime1'].dt.hour

pandora3['time']= pandora3['Hour']+pandora3['Day']*100+pandora3['Month']*10000+pandora3['Year']*1000000
pandora3['time']= pandora3['time'].astype(int)
pandora3['time']= pandora3['time'].astype(str)

pandora3['ID'] = pandora3['time']+' '+pandora3['Location']

pandora3

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_62208/2755706391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora3['Year'] = pandora3['Datetime1'].dt.year
/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_62208/2755706391.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandora3['Month'] = pandora3['Datetime1'].dt.month
/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_62208/2755706391.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,VCD(moles/m2),Location,Lat,Lon,Datetime1,Year,Month,Day,Hour,time,ID
38314,0.000186,Seoul-SNU,37.4580,126.951,2021-01-03 02:00:00,2021,1,3,2,2021010302,2021010302 Seoul-SNU
38478,0.000173,Seoul-SNU,37.4580,126.951,2021-01-06 02:00:00,2021,1,6,2,2021010602,2021010602 Seoul-SNU
38479,0.000172,Seoul-SNU,37.4580,126.951,2021-01-06 02:00:00,2021,1,6,2,2021010602,2021010602 Seoul-SNU
38485,0.000178,Seoul-SNU,37.4580,126.951,2021-01-06 03:00:00,2021,1,6,3,2021010603,2021010603 Seoul-SNU
38486,0.000177,Seoul-SNU,37.4580,126.951,2021-01-06 03:00:00,2021,1,6,3,2021010603,2021010603 Seoul-SNU
...,...,...,...,...,...,...,...,...,...,...,...
58325,0.000131,Tsukuba-NIES,36.0513,140.121,2023-12-30 03:00:00,2023,12,30,3,2023123003,2023123003 Tsukuba-NIES
58326,0.000130,Tsukuba-NIES,36.0513,140.121,2023-12-30 03:00:00,2023,12,30,3,2023123003,2023123003 Tsukuba-NIES
58332,0.000151,Tsukuba-NIES,36.0513,140.121,2023-12-30 04:00:00,2023,12,30,4,2023123004,2023123004 Tsukuba-NIES
58334,0.000159,Tsukuba-NIES,36.0513,140.121,2023-12-30 04:00:00,2023,12,30,4,2023123004,2023123004 Tsukuba-NIES


In [19]:
pandora4 = pandora3.groupby(['ID']).median()
# pandora4['time']= pandora4['Hour']+pandora4['Day']*100+pandora4['Month']*10000+pandora4['Year']*1000000
# pandora4['time']= pandora4['time'].astype(int)
# pandora4['time']= pandora4['time'].astype(str)
pandora4 = pandora4.reset_index()
pandora4[['time', 'loca']] = pandora4['ID'].str.split(' ', n=1, expand=True)

datetimeLis=[]
for j in pandora4['time']:

        datetime_obj = datetime.datetime(int(j[:4]), int(j[4:6]), int(j[6:8]), int(j[8:10]))
        datetime_obj = datetime_obj - timedelta(minutes=15)
        datetimeLis.append(datetime_obj)
pandora4['datetime']=datetimeLis
pandora4['Year'] = pandora4['datetime'].dt.year
pandora4['Month'] = pandora4['datetime'].dt.month
pandora4['Day'] = pandora4['datetime'].dt.day
pandora4['Hour'] = pandora4['datetime'].dt.hour
pandora4['time']= pandora4['Hour']+pandora4['Day']*100+pandora4['Month']*10000+pandora4['Year']*1000000

pandora4 = pandora4[[ 'VCD(moles/m2)', 'Lat', 'Lon', 'Hour',
       'time', 'loca', 'datetime']]
pandora4 = pandora4[pandora4['Hour'].isin([0,1,2,3,4,5,6,7,22,23])]
pandora4

/var/folders/4y/knznw84s4p5d2d77tb6wyq6h0000gn/T/ipykernel_62208/3702788574.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pandora4 = pandora3.groupby(['ID']).median()


,VCD(moles/m2),Lat,Lon,Hour,time,loca,datetime
0,0.000041,35.3207,139.6508,3,2021010103,Yokosuka,2021-01-01 03:45:00
1,0.000081,36.7769,126.4938,0,2021010200,Seosan,2021-01-02 00:45:00
2,0.000092,36.7769,126.4938,1,2021010201,Seosan,2021-01-02 01:45:00
3,0.000070,36.7769,126.4938,2,2021010202,Seosan,2021-01-02 02:45:00
4,0.000047,35.3207,139.6508,2,2021010202,Yokosuka,2021-01-02 02:45:00
...,...,...,...,...,...,...,...
31106,0.000195,35.6200,139.3834,4,2023123004,Tokyo-TMU,2023-12-30 04:45:00
31107,0.000467,23.7284,90.3982,6,2023123006,Dhaka,2023-12-30 06:45:00
31109,0.000173,35.6200,139.3834,1,2023123101,Tokyo-TMU,2023-12-31 01:45:00
31110,0.000186,35.6200,139.3834,2,2023123102,Tokyo-TMU,2023-12-31 02:45:00


In [20]:
pandora4 = pandora4.sort_values('time')

In [21]:
pandora4.to_pickle('Pandora_fus5')